In [1]:
import pymysql
import json
from functools import partial
import csv
import networkx as nx
import freeman as fm
from random import random, choices
def run_db_query(connection, query, args=None):
    with connection.cursor() as cursor:
        print('Executando query:')
        cursor.execute(query, args)
        for result in cursor:
            print(result)
            
def return_db_results(connection, query, args=None):#Gambiarra para pegar o id
    with connection.cursor() as cursor:
        cursor.execute(query, args)
        for result in cursor:
            return result #retorna logo o primeiro

def multiple_db_results(connection, query, args=None):#Gambiarra para pegar o id
    with connection.cursor() as cursor:
        cursor.execute(query, args)
        res = cursor.fetchall()
        return res

connection = pymysql.connect(
    host='localhost',
    user='root',
    password='vlm1998',
    database='redes')

db = partial(run_db_query, connection)

In [2]:
def add_node(g, n):
    g.add_node(n)
    g.nodes[n]['pos'] = (random(), random())
    g.nodes[n]['label'] = n
    g.nodes[n]['color'] = (255, 0, 0)

def add_edge(g, n, m, peso):
    #print("De: ", n, "para: ", m, "peso: ", peso)
    g.add_edge(n, m)
    g[n][m]['weight'] = peso

def get_team_name(connection, id_team):
    with connection.cursor() as cursor:
        cursor.execute("SELECT nome FROM times WHERE id_time = %s", (id_team))
        res = cursor.fetchone()
        if res:
            return res
        else:
            return -1
def get_team_id(connection, nome):
    with connection.cursor() as cursor:
        cursor.execute("SELECT id_time FROM times WHERE nome=%s", (nome))
        res = cursor.fetchone()
        if res:
            return res
        else:
            return -1
def list_all_teams(connection):
    with connection.cursor() as cursor:
        cursor.execute("SELECT nome FROM times")
        res = cursor.fetchall()
        return res
def list_all_transfers(connection, ano):
    with connection.cursor() as cursor:
        cursor.execute("SELECT * FROM transferencia WHERE ano=%s", (ano))
        res = cursor.fetchall()
        return res
def get_from_where(connection, id_jogador, ano):
    with connection.cursor() as cursor:
        cursor.execute("SELECT times.nome FROM waived INNER JOIN times USING(id_time) WHERE id_jogador = %s AND ano = %s ", (id_jogador, ano))
        res = cursor.fetchone()
        if res:
            return res
        else:
            return -1

def get_weight(connection, id_time1, id_time2, ano):
    with connection.cursor() as cursor:
        cursor.execute("SELECT peso FROM rivalidade WHERE id_time1=%s AND id_time2=%s AND ano=%s", (id_time1, id_time2, ano))
        res = cursor.fetchall()
        if res:
            return res
        else: #Pode ser que os nomes estejam invertidos nao sei
            cursor.execute("SELECT peso FROM rivalidade WHERE id_time1=%s AND id_time2=%s AND ano=%s", (id_time2, id_time1, ano))
            if res:
                return res
            return -1


In [3]:
a = fm.Animation()
g = nx.DiGraph()
a.rec(g)

transfers = list_all_transfers(connection, 2019)
teams = list_all_teams(connection)

for team in teams:
    add_node(g, team)

    
#Existem casos em que um time assina com um jogador e na mesma temporada vende ele, vamos ignorar estes casos por enquanto
for transfer in transfers:
    antigo = get_from_where(connection, transfer[1], transfer[3])
    novo = get_team_name(connection, transfer[2])
    if(antigo != -1) and (novo != -1):
        if(antigo != novo):
            id_antigo = get_team_id(connection, antigo)
            id_novo = get_team_id(connection, novo)
            weight = get_weight(connection, id_antigo, id_novo, 2019)
            if(weight == -1):
                weight = 0
            add_edge(g, antigo, novo, weight)
    else:
        print(antigo, novo)
    
    
a.rec(g)
a.play()

-1 ('Phoenix Suns',)
-1 ('Phoenix Suns',)
-1 ('New Orleans Pelicans',)
-1 ('Orlando Magic',)
-1 ('Milwaukee Bucks',)
-1 ('Sacramento Kings',)
-1 ('New York Knicks',)
-1 ('Oklahoma City Thunder',)
-1 ('Minnesota Timberwolves',)
-1 ('Milwaukee Bucks',)
-1 ('Indiana Pacers',)
-1 ('Utah Jazz',)
-1 ('Golden State Warriors',)
-1 ('New York Knicks',)
-1 ('Sacramento Kings',)
-1 ('Sacramento Kings',)
-1 ('Sacramento Kings',)
-1 ('Philadelphia 76ers',)
-1 ('Philadelphia 76ers',)
-1 ('New York Knicks',)
-1 ('Philadelphia 76ers',)
-1 ('Golden State Warriors',)
-1 ('Golden State Warriors',)
-1 ('Memphis Grizzlies',)
-1 ('Memphis Grizzlies',)
-1 ('Memphis Grizzlies',)
-1 ('Milwaukee Bucks',)
-1 ('Minnesota Timberwolves',)
-1 ('Minnesota Timberwolves',)
-1 ('Denver Nuggets',)
-1 ('Atlanta Hawks',)
-1 ('Golden State Warriors',)
-1 ('Houston Rockets',)
-1 ('Brooklyn Nets',)
-1 ('Memphis Grizzlies',)
-1 ('Brooklyn Nets',)
-1 ('Los Angeles Lakers',)
-1 ('Miami Heat',)
-1 ('Los Angeles Lakers',)
-1 ('Min

In [4]:


a = fm.Animation()
g = nx.DiGraph()
a.rec(g)

transfers = list_all_transfers(connection, 2018)
teams = list_all_teams(connection)

for team in teams:
    add_node(g, team)

    
#Existem casos em que um time assina com um jogador e na mesma temporada vende ele, vamos ignorar estes casos por enquanto
for transfer in transfers:
    antigo = get_from_where(connection, transfer[1], transfer[3])
    novo = get_team_name(connection, transfer[2])
    if(antigo != -1) and (novo != -1):
        if(antigo != novo):
            id_antigo = get_team_id(connection, antigo)
            id_novo = get_team_id(connection, novo)
            weight = get_weight(connection, id_antigo, id_novo, 2018)
            if(weight == -1):
                weight = 0
            add_edge(g, antigo, novo, weight)
    else:
        print(antigo, novo)
    
    
a.rec(g)
a.play()


-1 ('Chicago Bulls',)
-1 ('Phoenix Suns',)
-1 ('Los Angeles Lakers',)
-1 ('Sacramento Kings',)
-1 ('Orlando Magic',)
-1 ('Dallas Mavericks',)
-1 ('Memphis Grizzlies',)
-1 ('Memphis Grizzlies',)
-1 ('New York Knicks',)
-1 ('Detroit Pistons',)
-1 ('Atlanta Hawks',)
-1 ('Houston Rockets',)
-1 ('Milwaukee Bucks',)
-1 ('Golden State Warriors',)
-1 ('Phoenix Suns',)
-1 ('New Orleans Pelicans',)
-1 ('San Antonio Spurs',)
-1 ('Portland Trail Blazers',)
-1 ('Brooklyn Nets',)
-1 ('New Orleans Pelicans',)
-1 ('New Orleans Pelicans',)
-1 ('Philadelphia 76ers',)
-1 ('Chicago Bulls',)
-1 ('Boston Celtics',)
-1 ('Houston Rockets',)
-1 ('Atlanta Hawks',)
-1 ('Indiana Pacers',)
-1 ('Sacramento Kings',)
-1 ('Cleveland Cavaliers',)
-1 ('Boston Celtics',)
-1 ('Brooklyn Nets',)
-1 ('Phoenix Suns',)
-1 ('Phoenix Suns',)
-1 ('Washington Wizards',)
-1 ('Atlanta Hawks',)
-1 ('Dallas Mavericks',)
-1 ('Dallas Mavericks',)
-1 ('Dallas Mavericks',)
-1 ('New York Knicks',)
-1 ('Los Angeles Lakers',)
-1 ('Los Angele

In [5]:
a = fm.Animation()
g = nx.DiGraph()
a.rec(g)

transfers = list_all_transfers(connection, 2017)
teams = list_all_teams(connection)

for team in teams:
    add_node(g, team)

    
#Existem casos em que um time assina com um jogador e na mesma temporada vende ele, vamos ignorar estes casos por enquanto
for transfer in transfers:
    antigo = get_from_where(connection, transfer[1], transfer[3])
    novo = get_team_name(connection, transfer[2])
    if(antigo != -1) and (novo != -1):
        if(antigo != novo):
            id_antigo = get_team_id(connection, antigo)
            id_novo = get_team_id(connection, novo)
            weight = get_weight(connection, id_antigo, id_novo, 2018)
            if(weight == -1):
                weight = 0
            add_edge(g, antigo, novo, weight)
    else:
        print(antigo, novo)
    
    
a.rec(g)
a.play()

-1 ('Brooklyn Nets',)
-1 ('Brooklyn Nets',)
-1 ('Brooklyn Nets',)
-1 ('New Orleans Pelicans',)
-1 ('Atlanta Hawks',)
-1 ('New York Knicks',)
-1 ('Boston Celtics',)
-1 ('New York Knicks',)
-1 ('Dallas Mavericks',)
-1 ('Orlando Magic',)
-1 ('Indiana Pacers',)
-1 ('Minnesota Timberwolves',)
-1 ('Orlando Magic',)
-1 ('Los Angeles Lakers',)
-1 ('Sacramento Kings',)
-1 ('Charlotte Hornets',)
-1 ('Memphis Grizzlies',)
-1 ('New Orleans Pelicans',)
-1 ('Oklahoma City Thunder',)
-1 ('Indiana Pacers',)
-1 ('Atlanta Hawks',)
-1 ('Los Angeles Lakers',)
-1 ('Memphis Grizzlies',)
-1 ('Oklahoma City Thunder',)
-1 ('Dallas Mavericks',)
-1 ('Dallas Mavericks',)
-1 ('Detroit Pistons',)
-1 ('San Antonio Spurs',)
-1 ('Minnesota Timberwolves',)
-1 ('Cleveland Cavaliers',)
-1 ('Philadelphia 76ers',)
-1 ('Dallas Mavericks',)
-1 ('Dallas Mavericks',)
-1 ('Miami Heat',)
-1 ('Boston Celtics',)
-1 ('Sacramento Kings',)
-1 ('Chicago Bulls',)
-1 ('New Orleans Pelicans',)
-1 ('Utah Jazz',)
-1 ('Philadelphia 76ers',)

In [6]:
a = fm.Animation()
g = nx.DiGraph()
a.rec(g)

transfers = list_all_transfers(connection, 2016)
teams = list_all_teams(connection)

for team in teams:
    add_node(g, team)

    
#Existem casos em que um time assina com um jogador e na mesma temporada vende ele, vamos ignorar estes casos por enquanto
for transfer in transfers:
    antigo = get_from_where(connection, transfer[1], transfer[3])
    novo = get_team_name(connection, transfer[2])
    if(antigo != -1) and (novo != -1):
        if(antigo != novo):
            id_antigo = get_team_id(connection, antigo)
            id_novo = get_team_id(connection, novo)
            weight = get_weight(connection, id_antigo, id_novo, 2018)
            if(weight == -1):
                weight = 0
            add_edge(g, antigo, novo, weight)
    else:
        print(antigo, novo)
    
    
a.rec(g)
a.play()

-1 ('Sacramento Kings',)
-1 ('Minnesota Timberwolves',)
-1 ('Cleveland Cavaliers',)
-1 ('LA Clippers',)
-1 ('Houston Rockets',)
-1 ('New York Knicks',)
-1 ('Orlando Magic',)
-1 ('Orlando Magic',)
-1 ('Orlando Magic',)
-1 ('LA Clippers',)
-1 ('LA Clippers',)
-1 ('LA Clippers',)
-1 ('New York Knicks',)
-1 ('Phoenix Suns',)
-1 ('Sacramento Kings',)
-1 ('LA Clippers',)
-1 ('Philadelphia 76ers',)
-1 ('Houston Rockets',)
-1 ('Orlando Magic',)
-1 ('Brooklyn Nets',)
-1 ('Indiana Pacers',)
-1 ('Dallas Mavericks',)
-1 ('Chicago Bulls',)
-1 ('Golden State Warriors',)
-1 ('Boston Celtics',)
-1 ('Boston Celtics',)
-1 ('Dallas Mavericks',)
-1 ('San Antonio Spurs',)
-1 ('New Orleans Pelicans',)
-1 ('New Orleans Pelicans',)
-1 ('New Orleans Pelicans',)
-1 ('Atlanta Hawks',)
-1 ('Los Angeles Lakers',)
-1 ('Phoenix Suns',)
-1 ('Golden State Warriors',)
-1 ('Miami Heat',)
-1 ('Portland Trail Blazers',)
-1 ('LA Clippers',)
-1 ('Dallas Mavericks',)
-1 ('San Antonio Spurs',)
-1 ('Brooklyn Nets',)
-1 ('New Y

In [7]:
## A rede de 2015 será ignorada pois precisaria das informações de waived do ano anterior para estar mais completa

a = fm.Animation()
g = nx.DiGraph()
a.rec(g)

transfers = list_all_transfers(connection, 2015)
teams = list_all_teams(connection)

for team in teams:
    add_node(g, team)

    
#Existem casos em que um time assina com um jogador e na mesma temporada vende ele, vamos ignorar estes casos por enquanto
for transfer in transfers:
    antigo = get_from_where(connection, transfer[1], transfer[3])
    novo = get_team_name(connection, transfer[2])
    if(antigo != -1) and (novo != -1):
        if(antigo != novo):
            id_antigo = get_team_id(connection, antigo)
            id_novo = get_team_id(connection, novo)
            weight = get_weight(connection, id_antigo, id_novo, 2018)
            if(weight == -1):
                weight = 0
            add_edge(g, antigo, novo, weight)
    #else:
        #print(antigo, novo)
    
    
a.rec(g)
a.play()

In [8]:
connection.close()